In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from pydantic import BaseModel
from datetime import datetime
from pprint import pprint

In [23]:
class Scene(BaseModel):
    name: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]

class Scenes(BaseModel):
    scenes: list[Scene]

scenes_list = ["출퇴근길", "휴식", "공부", "게임", "유튜브 시청", "애완동물 돌보기"]

In [4]:
class User(BaseModel):
    name: str
    location: str
    birthdate: datetime
    occupation: str
    personality: list[str]
    scenes: list[Scene] = []
    prompt: str
    positives: list[str]
    negatives: list[str]
    
user_context = {
    'name': '윤형석',
    'location': '서울',
    'birthdate': datetime.strptime('1990-03-28', '%Y-%m-%d'),
    'occupation': '예비창업자',
    'personality': ['Introverted', 'Intuitive', 'Thinking', 'Perceiving'],
    'prompt': 'Your prompt here',
    'positives': ['지적 호기심', '창의력', '직관성'],
    'negatives': ['계획성 부족', '조직력 부족', '무기력함']
}

user = User(**user_context)

user_bio = user.model_dump_json()

In [5]:
import json

pydantic_parser = PydanticOutputParser(pydantic_object=Scenes)
scene_parser = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
scene_prompt = ChatPromptTemplate.from_template("""
다음은 사용자의 하루를 구성하는 장면들입니다.
사용자의 정보와 이 장면들을 바탕으로, 각각의 장면에 어울리는 시간, 공간, 기타 태그를 3~5개씩 필요에 따라 작성하고, 아래 지침에 따라 json 포맷으로 반환하세요.

태그를 붙이는 목적은 할 일을 관리하기 위한 데이터베이스에 사용하기 위해서입니다.
각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.

시간 태그에는 휴일 여부, 요일, 하루 중의 시간대 등의 정보를 포함하세요.
공간 태그에는 사용자의 위치, 활동하는 장소 등의 정보를 포함하세요.
기타 태그에는 시간과 공간 태그에 포함되지 않지만 할 일의 맥락과 상황을 검색하기에 좋은 정보를 포함하세요.

사용자 정보: {bio}
장면: {scenes}
지침: {format_instruction}
""").partial(format_instruction=pydantic_parser.get_format_instructions())

scenes_chain = scene_prompt | scene_parser | pydantic_parser
response = scenes_chain.invoke({"bio": user_bio, "scenes": scenes_list})

for scene in response.scenes:
    user.scenes.append(scene)
    print(scene.model_dump_json())

{"name":"출퇴근길","location_tags":["서울","지하철","버스"],"time_tags":["주중","오전","오후"],"other_tags":["교통","스트레스","사람들"]}
{"name":"휴식","location_tags":["집","카페"],"time_tags":["주말","오후","저녁"],"other_tags":["휴식","재충전","혼자"]}
{"name":"공부","location_tags":["집","도서관"],"time_tags":["주중","오전","오후"],"other_tags":["집중","자기계발","지식"]}
{"name":"게임","location_tags":["집"],"time_tags":["주말","저녁","심야"],"other_tags":["오락","스트레스 해소","혼자"]}
{"name":"유튜브 시청","location_tags":["집","휴대폰"],"time_tags":["주중","저녁","심야"],"other_tags":["정보","오락","취미"]}
{"name":"애완동물 돌보기기","location_tags":["집"],"time_tags":["주중","오전","오후"],"other_tags":["책임감","사랑","스트레스 해소"]}


In [6]:
def collect_tags(user):
    location_tags = []
    time_tags = []
    other_tags = []

    for scene in user.scenes:
        location_tags += scene.location_tags
        time_tags += scene.time_tags
        other_tags += scene.other_tags

    location_tags = list(set(location_tags))
    time_tags = list(set(time_tags))
    other_tags = list(set(other_tags))

    return location_tags, time_tags, other_tags

collect_tags(user)

(['버스', '카페', '도서관', '집', '지하철', '휴대폰', '서울'],
 ['주말', '저녁', '오후', '심야', '오전', '주중'],
 ['스트레스',
  '사랑',
  '집중',
  '지식',
  '재충전',
  '오락',
  '정보',
  '취미',
  '책임감',
  '교통',
  '스트레스 해소',
  '사람들',
  '휴식',
  '혼자',
  '자기계발'])

In [7]:
from langchain.docstore.document import Document

def store_tags(tag_list, tag_category):
    documents = []
    
    for i, tag in enumerate(tag_list):
        document = Document(page_content=tag, metadata={'source': f'{tag_category}_{i+1}'})
        documents.append(document)

    vectorstore = Chroma.from_documents(
        documents = documents,
        embedding = OpenAIEmbeddings(model = 'text-embedding-3-large'),
        collection_name = f'{tag_category}_tags',
        persist_directory = 'data'
    )
    
    return vectorstore

location_tag_vectors = store_tags(collect_tags(user)[0], 'location')
time_tag_vectors = store_tags(collect_tags(user)[1], 'time')
other_tag_vectors = store_tags(collect_tags(user)[2], 'other')

In [21]:
def print_answer_to_query(query, vectorstore, show_all = False):
    try:
        if vectorstore._collection.count() == 0:
            print("벡터 저장소가 비어 있습니다.")
            return
            
        print(f'벡터 저장소에 저장된 태그의 수: {vectorstore._collection.count()}')
        results = vectorstore.similarity_search_with_score(query)
        
        if results:
            print(f"검색 결과의 길이: {len(results)}")
            print(f"가장 유사한 결과: {results[0][0].page_content} (벡터 거리: {results[0][1]:.4f})\n")

            if show_all:
                for i, (doc, score) in enumerate(results):
                    print(f"결과 {i + 1}: {doc.page_content}")
                    print(f"벡터 거리: {score:.4f}\n")
        else:
            print("검색 결과가 없습니다.")
            
    except Exception as e:
        print(f"오류 발생: {e}")

print_answer_to_query("강아지", other_tag_vectors, show_all=True)

벡터 저장소에 저장된 태그의 수: 15
검색 결과의 길이: 4
가장 유사한 결과: 사랑 (벡터 거리: 1.2845)

결과 1: 사랑
벡터 거리: 1.2845

결과 2: 혼자
벡터 거리: 1.3265

결과 3: 교통
벡터 거리: 1.3824

결과 4: 사람들
벡터 거리: 1.3853



In [22]:
import json

user_bio = user.model_dump_json()
user_dict = json.loads(user_bio)
user_dict['scenes'][5]

{'name': '애완동물 돌보기기',
 'location_tags': ['집'],
 'time_tags': ['주중', '오전', '오후'],
 'other_tags': ['책임감', '사랑', '스트레스 해소']}

In [10]:
# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=None)

In [11]:
from langchain.schema import BaseOutputParser

class CustomListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> list[str]:
        responses = text.split("---")
        items = [response.strip() for response in responses]
        return items

    def get_format_instructions(self) -> str:
        return '서로 다른 답변은 "---"로 구분하세요.'

# 파서 초기화
response_parser = CustomListOutputParser()

In [12]:
prompt_template = ChatPromptTemplate.from_template("""
다음은 사용자 정보입니다. 이 정보를 바탕으로, 사용자의 성격과 하루 일과, 주요 관심사를를 상상해서 1문단으로 작성하세요.
이를 작성하는 이유는 사용자의 할 일을 사용자의 생활패턴과 맥락에 맞게 구체화하여 추천하기 위해서입니다.
사용자에 대한 이해가 깊어질수록 사용자에게 더 유용한 할 일을 추천할 수 있습니다.
사용자의 긍정적인 면과 부정적인 면을 모두 포함할 수 있도록 작성하세요.

작성된 내용 중 사용자가 적합한 것을 선택할 수 있도록, 서로 다른 내용의 답변을 3~5개 생성하세요.
각각의 답변은 사용자 정보의 다른 부분에 집중하며, 서로 비슷하지 않은 내용이어야 합니다.
예를 들어 한 답변이 "대중교통"이라는 키워드에 집중한다면, 다른 답변은 "도서관" 등 다른 맥락에 집중할 수 있습니다.
만약 비슷한 답변이 생성된다면 생략하세요.

{format_instruction}

사용자 정보: {bio}

답변:
""").partial(format_instruction=response_parser.get_format_instructions())

chain = prompt_template | llm | response_parser
responses = chain.invoke({"bio": user_bio})

In [13]:
for response in responses:
    print(response)
    print("---")

윤형석은 서울에 거주하는 예비창업자로, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 지적 호기심이 강해 새로운 지식을 탐구하는 것을 즐기지만, 때때로 계획성과 조직력이 부족해 목표 달성에 어려움을 겪기도 합니다. 주중에는 지하철이나 버스를 이용해 출퇴근하며, 이 과정에서 사람들로 인한 스트레스를 느끼기도 합니다. 주말에는 집이나 카페에서 혼자 휴식을 취하며 재충전하는 시간을 가지며, 저녁에는 게임이나 유튜브 시청으로 오락을 즐깁니다. 또한, 애완동물을 돌보는 일에서 책임감을 느끼며 사랑을 쏟고, 이를 통해 스트레스를 해소하기도 합니다.
---
윤형석은 서울에서 예비창업자로 활동하며, 지적 호기심이 강한 내향적인 성격을 가지고 있습니다. 그는 주중에 집이나 도서관에서 집중적으로 공부하며 자기계발에 힘쓰고, 새로운 아이디어를 구상하는 데 많은 시간을 할애합니다. 그러나 계획성이 부족해 가끔은 목표를 잃고 무기력해지기도 합니다. 주말에는 혼자서 카페에서 책을 읽거나, 집에서 게임을 하며 스트레스를 해소하는 시간을 가지며, 이러한 혼자만의 시간에서 창의력을 발휘하기도 합니다.
---
윤형석은 서울에 거주하는 예비창업자로, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 주중에 출퇴근길에 지하철이나 버스를 이용하며, 이 과정에서 느끼는 스트레스를 해소하기 위해 주말에는 집에서 애완동물을 돌보는 일에 많은 시간을 할애합니다. 애완동물과의 교감은 그에게 큰 사랑과 책임감을 느끼게 하며, 이를 통해 무기력함을 극복하는 데 도움을 줍니다. 또한, 저녁 시간에는 유튜브를 시청하며 정보를 얻고 오락을 즐기는 것을 좋아합니다.
---
윤형석은 서울에서 예비창업자로 활동하며, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 주중에 집에서 공부하며 자기계발에 힘쓰고, 지적 호기심을 충족시키기 위해 다양한 자료를 탐색합니다. 그러나 계획성과 조직력이 부족해 가끔은 목표를 잃고 무기력해지기도 합니다. 주말에는 혼자서 카페에서 재충전하는 시간을 가지며, 저녁에는 게임을 통해 스트레

In [14]:
def set_prompt(user: User, responses: list[str], index: int):
    if index < 0 or index >= len(responses):
        raise IndexError("Index out of bounds.")
    user.prompt = responses[index].strip()

set_prompt(user, responses, 3)
user.prompt

'윤형석은 서울에서 예비창업자로 활동하며, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 주중에 집에서 공부하며 자기계발에 힘쓰고, 지적 호기심을 충족시키기 위해 다양한 자료를 탐색합니다. 그러나 계획성과 조직력이 부족해 가끔은 목표를 잃고 무기력해지기도 합니다. 주말에는 혼자서 카페에서 재충전하는 시간을 가지며, 저녁에는 게임을 통해 스트레스를 해소하는 것을 즐깁니다. 이러한 혼자만의 시간은 그에게 창의력을 발휘할 수 있는 기회를 제공합니다.'

In [15]:
class Task(BaseModel):
    name: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]
    estimated_minutes: int
    
class Tasks(BaseModel):
    name: str
    context: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]
    tasks: list[Task]

tasks_pydantic_parser = PydanticOutputParser(pydantic_object=Tasks)

In [16]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()

task_breakdown = ChatPromptTemplate.from_template("""
다음은 사용자가 입력한 헤야 할 일입니다. 이 할 일을 세부적으로 나누어 작성하세요.
각각의 할 일은 최대한 구체적인 행동으로 작성하고, 이를 수행하는 데 필요한 시간과 노력을 고려하여 작성하세요.
할 일을 세부적으로 나누면 사용자가 할 일을 더 쉽게 완료할 수 있습니다.

각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.
각각의 할 일은 사용자의 위치, 활동하는 장소, 시간대 등을 고려하여 태그를 부여하세요.

Tasks의 context에서는 사용자 정보와 하루 일과에 기반하여 해당 할 일을 언제 하는 것이 좋을지 설명하세요.
각각의 세부 할 일에 적절한 태그를 부여하세요.

{format_instruction}

사용자의 인적 정보: {bio}
사용자의 하루 일과: {prompt}
사용자가 입력한 할 일: {task}
""").partial(format_instruction=tasks_pydantic_parser.get_format_instructions())

task_chain = task_breakdown | llm
task_response = task_chain.invoke({"bio": user_bio, "prompt": user.prompt, "task": "체중 감량을 위한 계획 및 실천하기"})

In [17]:
print(task_response.content)

```json
{
  "name": "체중 감량을 위한 계획 및 실천하기",
  "context": "윤형석은 체중 감량을 위해 구체적인 계획을 세우고 이를 실천해야 합니다. 주중에 공부와 자기계발을 하면서도 건강을 챙길 수 있는 방법을 찾아야 합니다.",
  "location_tags": ["집", "헬스장", "카페"],
  "time_tags": ["주중", "오전", "오후", "저녁"],
  "other_tags": ["건강", "자기계발", "스트레스 해소"],
  "tasks": [
    {
      "name": "체중 감량 목표 설정하기",
      "location_tags": ["집"],
      "time_tags": ["주중", "오후"],
      "other_tags": ["건강", "계획"],
      "estimated_minutes": 30
    },
    {
      "name": "주간 운동 계획 세우기",
      "location_tags": ["집"],
      "time_tags": ["주중", "오후"],
      "other_tags": ["건강", "계획"],
      "estimated_minutes": 30
    },
    {
      "name": "헬스장 등록 및 첫 운동하기",
      "location_tags": ["헬스장"],
      "time_tags": ["주중", "저녁"],
      "other_tags": ["건강", "운동"],
      "estimated_minutes": 60
    },
    {
      "name": "건강한 식단 계획하기",
      "location_tags": ["집"],
      "time_tags": ["주중", "오후"],
      "other_tags": ["건강", "식사"],
      "estimated_minutes": 45
    },
    {
      "name": "식사 일지 작성하기",
 